<a href="https://colab.research.google.com/github/borislevant/SciComPy/blob/master/cifar10_from_softmax_to_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will demonstrate the work on the CIFAR-10 data set from the simplest Softmax model (Logistic regression) to the more advanced methods

## 1. Introduction
The CIFAR-10 dataset contains 60,000 color images of 32 x 32 pixels in 3 channels divided into 10
classes. Each class contains 6,000 images. The training set contains 50,000 images, while the test sets
provides 10,000 images. This image taken from the CIFAR repository ( <a href = "https://www.cs.toronto.edu/~kriz/cifar.html">https://www.cs.toronto.edu/~kriz/cifar.html </a>). This is a classification problem with 10 classes(muti-label classification). We can take a view on this image for more comprehension of the dataset. 

![cifar10.png](https://github.com/borislevant/SciComPy/blob/master/cifar10.png?raw=1)


The challenge is to recognize previously unseen images and assign them to one of the 10 classes.

Ok Let's get started.

## 2. Import and Preprocess the data



### 2.1 Import all required libraries

In [15]:
from __future__ import print_function
from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, InputLayer
from keras.layers import Conv2D, MaxPooling2D
import os

import numpy as np

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import itertools

%matplotlib inline


In [16]:
batch_size = 32  # The default batch size of keras.
num_classes = 10  # Number of class for the dataset
epochs = 30
data_augmentation = False
opt = keras.optimizers.Adam(learning_rate=0.0001)
reg = keras.regularizers.l2(l2=0.0001)

### 2.2 Import and preproces of data 
We load the data and split it between train and test sets


In [17]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [18]:
# Normalize the data. Before we need to connvert data type to float for computation.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##3. Softmax model

In [19]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=reg))
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 3072)              0         
                                                                 
 dense_6 (Dense)             (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)


Epoch 1/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9787 - accuracy: 0.2968 - val_loss: 1.8738 - val_accuracy: 0.3378
Epoch 2/30
1563/1563 [==============================] - 4s 3ms/step - loss: 1.8507 - accuracy: 0.3540 - val_loss: 1.8281 - val_accuracy: 0.3647
Epoch 3/30
1563/1563 [==============================] - 4s 3ms/step - loss: 1.8172 - accuracy: 0.3690 - val_loss: 1.8119 - val_accuracy: 0.3690
Epoch 4/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7949 - accuracy: 0.3779 - val_loss: 1.8063 - val_accuracy: 0.3675
Epoch 5/30
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7808 - accuracy: 0.3864 - val_loss: 1.7892 - val_accuracy: 0.3754
Epoch 6/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7706 - accuracy: 0.3882 - val_loss: 1.7813 - val_accuracy: 0.3794
Epoch 7/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7615 - accuracy: 0.3935 - val_loss: 1.7960 - val_accuracy:

##4. Fully-connected Neural Network (FNN) model

In [21]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_regularizer=reg))
model.add(Dense(256, activation='relu', kernel_regularizer=reg))
model.add(Dense(64, activation='relu', kernel_regularizer=reg))
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=reg))
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 3072)              0         
                                                                 
 dense_7 (Dense)             (None, 1024)              3146752   
                                                                 
 dense_8 (Dense)             (None, 256)               262400    
                                                                 
 dense_9 (Dense)             (None, 64)                16448     
                                                                 
 dense_10 (Dense)            (None, 10)                650       
                                                                 
Total params: 3,426,250
Trainable params: 3,426,250
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Epoch 1/30
1563/1563 [==============================] - 8s 4ms/step - loss: 2.0209 - accuracy: 0.3193 - val_loss: 1.8547 - val_accuracy: 0.3819
Epoch 2/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.7894 - accuracy: 0.3932 - val_loss: 1.7113 - val_accuracy: 0.4220
Epoch 3/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.6852 - accuracy: 0.4270 - val_loss: 1.6475 - val_accuracy: 0.4346
Epoch 4/30
1563/1563 [==============================] - 7s 5ms/step - loss: 1.6200 - accuracy: 0.4471 - val_loss: 1.6188 - val_accuracy: 0.4373
Epoch 5/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.5731 - accuracy: 0.4645 - val_loss: 1.5833 - val_accuracy: 0.4625
Epoch 6/30
1563/1563 [==============================] - 7s 5ms/step - loss: 1.5373 - accuracy: 0.4768 - val_loss: 1.5382 - val_accuracy: 0.4743
Epoch 7/30
1563/1563 [==============================] - 7s 5ms/step - loss: 1.5062 - accuracy: 0.4860 - val_loss: 1.5409 - val_accuracy: